In [1]:
import sqlite3
import pandas as pd

db_path = "eng_subtitles_database.db"  # Replace with your database path
chunksize = 10000  # Adjust chunksize as needed

try:
    conn = sqlite3.connect(db_path)
    query = "SELECT num, name, content FROM zipfiles;"
    
    all_chunks = []
    for chunk_df in pd.read_sql_query(query, conn, chunksize=chunksize):
        all_chunks.append(chunk_df)

    if all_chunks:
        full_df = pd.concat(all_chunks, ignore_index=True)
        print(f"Loaded {len(full_df)} rows from the database.")
        # Perform operations on full_df here
        # print(full_df.head()) #example to print first rows
    else:
        print("No data loaded from the database.")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")
finally:
    if 'conn' in locals() and conn:
        conn.close()

Loaded 82498 rows from the database.


In [3]:
# Applying the above Function on the Entire Data

import zipfile
import io

count = 0

def decode_method(binary_data):
    global count
    # Decompress the binary data using the zipfile module
    # print(count, end=" ")
    count += 1
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Assuming there's only one file in the ZIP archive
            subtitle_content = zip_file.read(zip_file.namelist()[0])

    # Now 'subtitle_content' should contain the extracted subtitle content
    return subtitle_content.decode("latin-1")  # Assuming the content is UTF-8 encoded text

In [4]:
full_df["file_content"] = full_df["content"].apply(decode_method)

In [5]:
full_df.head()

,num,name,content,file_content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


In [7]:
import random

df = full_df.sample(frac=0.3, random_state=42)

In [8]:
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to C:\Users\prasad
[nltk_data]     jadhav\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to C:\Users\prasad
[nltk_data]     jadhav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\prasad
[nltk_data]     jadhav\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
def clean_subtitle(subtitle):
    
    # Remove timestamps
    clean_content = re.sub(r"\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}\n?", '', subtitle)

    # Remove other non-textual patterns
    clean_content = re.sub(r"<[^>]+>", '', clean_content)

    clean_content = re.sub(r"[^\w\s]", '', clean_content)

    clean_content = re.sub(r"[^\x00-\x7F]+", '', clean_content)

    clean_content = re.sub(r"\b\d+\s", '', clean_content)

    # Convert to lowercase
    clean_content = clean_content.lower()

    # Tokenize the subtitle content
    tokens = word_tokenize(clean_content)

    # Remove stopwords and lemmatize tokens
    stop_words = set(stopwords.words("english"))

    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens if word.lower() not in stop_words]

    # Join the filtered tokens back into a string
    clean_content = ' '.join(clean_tokens)

    return clean_content.strip()

In [10]:
df["cleaned_content"] = df["file_content"].apply(clean_subtitle)

In [11]:
df.head()

,num,name,content,file_content,cleaned_content
17262,9251120,maybe.this.time.(2014).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x89\x9a\x...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch...",watch video online opensubtitles free browser ...
7294,9211589,down.the.shore.s01.e10.and.justice.for.all.(19...,b'PK\x03\x04\x14\x00\x00\x00\x08\x007\x8f\x99V...,"1\r\n00:00:09,275 --> 00:00:11,876\r\n¶ Oh, I ...",oh know getting late dont wan na go home im hu...
47707,9380845,uncontrollably.fond.s01.e07.heartache.(2016).e...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x8f\x19\x...,"1\r\n00:00:07,140 --> 00:00:14,220\r\n<i>Timin...",timing subtitle uncontrollable lovebird team v...
29914,9301436,screen.two.s13.e04.the.precious.blood.(1996).e...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00[\xaa\x99V...,"1\r\n00:00:06,133 --> 00:00:08,900\r\n[etherea...",ethereal music apiopensubtitlesorg deprecated ...
54266,9408707,battlebots.(2015).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xf4<\x9aV...,"ï»¿1\r\n00:00:01,480 --> 00:00:03,570\r\n[Chri...",chris oh minibots yelling oh leave little bot ...


In [12]:
# Function to count tokens

def count_tokens(text):
    tokens = word_tokenize(text)
    return len(tokens)

# Iterate through data and count tokens for each file
for index, entry in df.iterrows():
    total_tokens = count_tokens(entry["cleaned_content"])
    print(f"File: {entry['name']}, Number of Tokens: {total_tokens}")

File: maybe.this.time.(2014).eng.1cd, Number of Tokens: 5335
File: down.the.shore.s01.e10.and.justice.for.all.(1992).eng.1cd, Number of Tokens: 1356
File: uncontrollably.fond.s01.e07.heartache.(2016).eng.1cd, Number of Tokens: 2425
File: screen.two.s13.e04.the.precious.blood.(1996).eng.1cd, Number of Tokens: 3588
File: battlebots.(2015).eng.1cd, Number of Tokens: 7743
File: csi.crime.scene.investigation.s08.e16.two.and.a.half.deaths.(2008).eng.1cd, Number of Tokens: 2998
File: royal.ashes.().eng.1cd, Number of Tokens: 5537
File: return.to.seoul.(2022).eng.1cd, Number of Tokens: 2503
File: idris.elba.king.of.speed.s01.e02.episode.1.2.(2013).eng.1cd, Number of Tokens: 3433
File: tooth.pari.when.love.bites.s01.e08.episode.1.8.(2023).eng.1cd, Number of Tokens: 1863
File: studio.one.s08.e30.the.arena.(1956).eng.1cd, Number of Tokens: 3623
File: love.life.s02.e10.epilogue.(2021).eng.1cd, Number of Tokens: 2011
File: scrubs.s03.e14.my.screwup.(2004).eng.1cd, Number of Tokens: 1416
File: devot

File: its.beautiful.now.s01.e44.episode.1.44.(2022).eng.1cd, Number of Tokens: 3976
File: northern.exposure.s05.e20.a.wing.and.a.prayer.(1994).eng.1cd, Number of Tokens: 5384
File: narcos.s02.e02.cambalache.(2016).eng.1cd, Number of Tokens: 2325
File: 1899.s01.e05.the.calling.(2022).eng.1cd, Number of Tokens: 38
File: reign.s02.e10.mercy.(2014).eng.1cd, Number of Tokens: 2071
File: fruits.basket.s03.e05.i.mean.you.know.right.(2021).eng.1cd, Number of Tokens: 1299
File: iki.wo.hisomete.s01.e03.episode.1.3.(2021).eng.1cd, Number of Tokens: 1014
File: beverly.hills.90210.s06.e08.gypsies.cramps.and.fleas.(1995).eng.1cd, Number of Tokens: 2333
File: scrubs.s09.e05.our.mysteries.(2009).eng.1cd, Number of Tokens: 1566
File: sexify.s02.e03.episode.2.3.(2023).eng.1cd, Number of Tokens: 1712
File: murdoch.mysteries.s04.e01.tattered.and.torn.(2011).eng.1cd, Number of Tokens: 2876
File: high.school.musical.the.musical.the.series.s03.e05.the.real.campers.of.shallow.lake.(2022).eng.1cd, Number of To

File: the.mulligan.(2022).eng.1cd, Number of Tokens: 6208
File: auggie.rose.(2000).eng.1cd, Number of Tokens: 4611
File: darby.and.joan.s01.e08.episode.1.8.(2022).eng.1cd, Number of Tokens: 2197
File: record.of.ragnarok.s02.e09.resonance.(2023).eng.1cd, Number of Tokens: 898
File: dallas.s01.e03.the.price.you.pay.(2012).eng.1cd, Number of Tokens: 2238
File: sound.of.silence.(2023).eng.1cd, Number of Tokens: 922
File: pepsi.wheres.my.jet.s01.e01.the.kid.from.seattle.(2022).eng.1cd, Number of Tokens: 64
File: magnum.p.i.s03.e03.no.way.out.(2020).eng.1cd, Number of Tokens: 2429
File: as.if.s02.e08.resimdeki.unlu.(2022).eng.1cd, Number of Tokens: 2486
File: break.point.s01.e03.california.dreaming.(2023).eng.1cd, Number of Tokens: 70
File: madam.secretary.s06.e10.leaving.the.station.(2019).eng.1cd, Number of Tokens: 3180
File: poldark.s05.e01.episode.5.1.(2019).eng.1cd, Number of Tokens: 2240
File: hannibal.(1959).eng.1cd, Number of Tokens: 3176
File: ray.donovan.s07.e07.the.transfer.agent.

File: northern.exposure.s03.e01.the.bumpy.road.to.love.(1991).eng.1cd, Number of Tokens: 5362
File: cash.register.s02.e18.fifty.million.(2020).eng.1cd, Number of Tokens: 1569
File: the.tick.s03.e05.devil.in.diapers.(1996).eng.1cd, Number of Tokens: 1205
File: love.triangle.(au).s01.e08.cinderellas.shoe.(2022).eng.1cd, Number of Tokens: 4723
File: butterflies.(2018).eng.1cd, Number of Tokens: 5778
File: inside.man.s01.e01.episode.1.1.(2022).eng.1cd, Number of Tokens: 3621
File: home.economics.s03.e02.melatonin.10.mg.tablets.14.99.(2022).eng.1cd, Number of Tokens: 1957
File: de.verraders.s02.e01.episode.2.1.(2021).eng.1cd, Number of Tokens: 2541
File: the.dresden.files.s01.e09.the.other.dick.(2007).eng.1cd, Number of Tokens: 2655
File: deinfluencer.(2022).eng.1cd, Number of Tokens: 4153
File: byeoleun.nae.gaseume.s01.e14.episode.1.14.(1997).eng.1cd, Number of Tokens: 1276
File: 8.out.of.10.cats.does.countdown.s05.e05.episode.7.6.(2015).eng.1cd, Number of Tokens: 3022
File: step.up.high.w

File: non.non.biyori.s02.e09.we.looked.at.the.moon.together.(2015).eng.1cd, Number of Tokens: 2245
File: cain.s01.e02.justices.(2012).eng.1cd, Number of Tokens: 1835
File: young.sheldon.s05.e20.uncle.sheldon.and.a.hormonal.firecracker.(2022).eng.1cd, Number of Tokens: 1260
File: marry.me.again.(2023).eng.1cd, Number of Tokens: 6134
File: todos.mienten.s01.e02.episodio.2.(2022).eng.1cd, Number of Tokens: 1951
File: taiwan.crime.stories.s01.e09.gravity.of.sin.3.(2023).eng.1cd, Number of Tokens: 2039
File: crazy.wedding.2.(2019).eng.1cd, Number of Tokens: 4940
File: run.silent.run.deep.(1958).eng.1cd, Number of Tokens: 4643
File: rabin.the.last.day.(2015).eng.1cd, Number of Tokens: 6445
File: american.dad.s04.e01.1600.candles.(2008).eng.1cd, Number of Tokens: 1550
File: beti.bete.(1964).eng.1cd, Number of Tokens: 4304
File: father.brown.s10.e05.the.hidden.man.(2023).eng.1cd, Number of Tokens: 2145
File: the.white.lotus.s02.e01.ciao.(2022).eng.1cd, Number of Tokens: 3582
File: the.middle.s

...

File: recess.s03.e03.dodgeball.city.(1999).eng.1cd, Number of Tokens: 1793
File: 24.s08.e16.day.8.700.a.m.800.a.m.(2010).eng.1cd, Number of Tokens: 2487
File: jackie.chan.adventures.s03.e04.aztec.rat.race.(2002).eng.1cd, Number of Tokens: 917
File: stargate.atlantis.s05.e14.the.prodigal.(2008).eng.1cd, Number of Tokens: 2367
File: medium.s01.e16.when.push.comes.to.shove.part.1.(2005).eng.1cd, Number of Tokens: 2557
File: close.to.home.murder.in.the.coalfield.s01.e03.neue.herren.(2022).eng.1cd, Number of Tokens: 27
File: marc.marquez.all.in.s01.e05.come.back.(2023).eng.1cd, Number of Tokens: 1934
File: leva.life.s01.e02.episode.1.2.().eng.1cd, Number of Tokens: 912
File: murdoch.mysteries.s01.e05.til.death.do.us.part.(2008).eng.1cd, Number of Tokens: 2947
File: empire.s06.e06.heart.of.stone.(2019).eng.1cd, Number of Tokens: 3104
File: young.hercules.s01.e50.valley.of.the.shadow.(1999).eng.1cd, Number of Tokens: 1327
File: the.grid.s01.e04.episode.1.4.(2004).eng.1cd, Number of Tokens: 21

File: mom.s08.e17.a.community.hero.and.a.wide.turn.(2021).eng.1cd, Number of Tokens: 2208
File: family.guy.s03.e09.mr.saturday.knight.(2001).eng.1cd, Number of Tokens: 1706
File: american.experience.s19.e15.alexander.hamilton.(2007).eng.1cd, Number of Tokens: 7293
File: once.upon.a.time.s06.e01.the.savior.(2016).eng.1cd, Number of Tokens: 1898
File: beverly.hills.90210.s03.e09.highwire.(1992).eng.1cd, Number of Tokens: 2915
File: bump.s01.e02.the.startle.reflex.(2021).eng.1cd, Number of Tokens: 1178
File: 90210.s01.e06.model.behavior.(2008).eng.1cd, Number of Tokens: 2951
File: murdoch.mysteries.s08.e03.glory.days.(2014).eng.1cd, Number of Tokens: 2644
File: two.and.a.half.men.s07.e12.fart.jokes.pie.and.celeste.(2010).eng.1cd, Number of Tokens: 1453
File: the.opposite.sex.(1956).eng.1cd, Number of Tokens: 7048
File: the.frighteners.(1996).eng.1cd, Number of Tokens: 4414
File: wu.gong.zhou.(1982).eng.1cd, Number of Tokens: 2358
File: mom.s03.e11.cinderella.and.a.drunk.macgyver.(2016).en

File: last.week.tonight.with.john.oliver.s10.e06.solitary.confinement.(2023).eng.1cd, Number of Tokens: 2965
File: the.gold.s01.e05.the.boy.you.were.(2023).eng.1cd, Number of Tokens: 2382
File: jackie.chan.adventures.s05.e13.the.powers.that.be.part.2.(2005).eng.1cd, Number of Tokens: 1354
File: love.after.music.s01.e08.el.perfume.que.lleva.el.dolor.(2023).eng.1cd, Number of Tokens: 1744
File: my.name.is.earl.s04.e05.sweet.johnny.(2008).eng.1cd, Number of Tokens: 1574
File: at.last.the.1948.show.s02.e04.episode.2.4.(1967).eng.1cd, Number of Tokens: 1838
File: king.of.the.hill.s05.e11.hank.and.the.great.glass.elevator.(2001).eng.1cd, Number of Tokens: 1305
File: the.abbott.and.costello.show.s02.e14.wife.wanted.(1954).eng.1cd, Number of Tokens: 1810
File: harlequin.(1980).eng.1cd, Number of Tokens: 3997
File: friends.s05.e02.the.one.with.all.the.kissing.(1998).eng.1cd, Number of Tokens: 1617
File: folklore.s02.e04.7.days.of.hell.(philippines).(2021).eng.1cd, Number of Tokens: 1409
File: m

File: jodha.akbar.s01.e309.episode.1.309.(2014).eng.1cd, Number of Tokens: 12979
File: young.lady.and.gentleman.s01.e17.episode.1.17.(2021).eng.1cd, Number of Tokens: 2890
File: bridge.of.the.doomed.(2022).eng.1cd, Number of Tokens: 3503
File: upstairs.downstairs.s02.e10.a.special.mischief.(1972).eng.1cd, Number of Tokens: 3166
File: friends.s07.e15.the.one.with.joeys.new.brain.(2001).eng.1cd, Number of Tokens: 1672
File: preacher.s01.e08.el.valero.(2016).eng.1cd, Number of Tokens: 1566
File: beverly.hills.90210.s03.e14.wild.horses.(1992).eng.1cd, Number of Tokens: 2996
File: dracula.(1974).eng.1cd, Number of Tokens: 1979
File: crayon.shinchan.shrouded.in.mystery.the.flowers.of.tenkazu.academy.(2021).eng.1cd, Number of Tokens: 4553
File: secrets.of.the.oligarch.wives.(2022).eng.1cd, Number of Tokens: 4941
File: endless.love.s01.e29.episode.1.29.(2016).eng.1cd, Number of Tokens: 5955
File: wednesday.s01.e07.if.you.dont.woe.me.by.now.(2022).eng.1cd, Number of Tokens: 2022
File: river.mon

File: masters.of.sex.s01.e06.brave.new.world.(2013).eng.1cd, Number of Tokens: 2938
File: khaleja.(2010).eng.1cd, Number of Tokens: 7626
File: beauty.and.the.beast.s03.e09.cats.out.of.the.bag.(2015).eng.1cd, Number of Tokens: 3053
File: 16.to.life.(2009).eng.1cd, Number of Tokens: 4463
File: ancient.unexplained.files.s01.e08.tomb.of.the.red.queen.(2021).eng.1cd, Number of Tokens: 2947
File: three.tall.women.(2022).eng.1cd, Number of Tokens: 6400
File: knots.landing.s02.e10.choices.(1981).eng.1cd, Number of Tokens: 2439
File: archie.bunkers.place.s03.e08.growing.up.is.hard.to.do.part.1.(1981).eng.1cd, Number of Tokens: 961
File: hua.jai.sila.s01.e04.episode.1.4.(2019).eng.1cd, Number of Tokens: 1504
File: married.with.children.s05.e24.route.666.part.2.(1991).eng.1cd, Number of Tokens: 1010
File: sapphire.steel.s03.e02.assignment.five.part.2.(1981).eng.1cd, Number of Tokens: 1378
File: super.giant.robot.brothers.s01.e02.welcome.to.g.r.i.t.s.(2022).eng.1cd, Number of Tokens: 1380
File: ma

In [13]:
# Define the document chunker function

def document_chunker(document, chunk_size=1500, overlap_size=200):
    tokens = word_tokenize(document)  # Tokenize the document
    num_tokens = len(tokens)
    chunks = []

    # Iterate over the tokens and create chunks with specified overlap
    for start in range(0, num_tokens, chunk_size - overlap_size):
        end = min(start + chunk_size, num_tokens)
        chunk = tokens[start:end]
        chunks.append({"name": row["name"], "chunk_index": len(chunks) + 1, "chunk_text": ' '.join(chunk)})

    return chunks

In [14]:
# Apply the document chunker to each cleaned document in the DataFrame

all_chunks = []
for index, row in df.iterrows():
    document = row["cleaned_content"]
    document_chunks = document_chunker(document)
    all_chunks.extend(document_chunks)

# Create a new DataFrame from the list of chunks
chunks_df = pd.DataFrame(all_chunks)

In [15]:
chunks_df.head()

,name,chunk_index,chunk_text
0,maybe.this.time.(2014).eng.1cd,1,watch video online opensubtitles free browser ...
1,maybe.this.time.(2014).eng.1cd,2,also answer question invest something trendy r...
2,maybe.this.time.(2014).eng.1cd,3,kitchen thats learned cook hmm thats miss moni...
3,maybe.this.time.(2014).eng.1cd,4,take ah wow like newlywed maybe time itll lovi...
4,maybe.this.time.(2014).eng.1cd,5,time maybe time love wont end two old friend m...


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
# Calculate TF-IDF for the documents

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(chunks_df["chunk_text"])

In [18]:
chunks_df["chunk_text"][27]

'father want understand needed go somewhere youve always loved japan tokyo imagine surprise got message seoul japan longer option could called taken moment talk talk told plane tokyo cancelled due typhoon told go two week holiday next flight offered company seoul took really way want tell always said would go together yes know uh think good came alone must go mom im running credit already take care write u okay yes give dad kiss yes promise love darling dont understand telegram thing send third telegram there still reply hammond send least one year would risk complaint harassment send someone registered letter even sure right address im sorry possible leave two day probably never come back want mother address understand im sorry id really like rule protect parent child ask miss benot wish send third telegram mother address whats wrong come see paris yes well go lot region ill introduce friend well walk favourite part paris id like go jardin de plantes okay well go father message messag

In [19]:
query = "father want understand needed go somewhere youve always loved japan tokyo imagine surprise got message seoul japan longer option could called taken moment talk talk told plane tokyo cancelled due typhoon told go two week holiday next flight offered company seoul took really way want tell always said would go together yes know uh think good came alone must go mom im running credit already take care write u okay yes give dad kiss yes promise love darling dont understand telegram thing send third telegram there still reply hammond send least one year would risk complaint harassment send someone registered letter even sure right address im sorry possible leave two day probably never come back want mother address understand im sorry id really like rule protect parent child ask miss benot wish send third telegram mother address whats wrong come see paris yes well go lot region ill introduce friend well walk favourite part paris id like go jardin de plantes okay well go father message message ive enough translate insult ill send shouldnt say freddie shouldnt write drive mad way communicating spewing sorrow way korean men im french youre also part korean freddie want stay korea stay korea go france want stay abandon kid ask become good little korean say understand music great tena gift tell dont want gift open boyfriend france boyfriend say even true tell like lot id like encounter become deep relationship tell freddie gift mean say love boyfriend france understand boyfriend stop freddie okay care hell get sulking youre leaving ill go pay little tena youre sad person yeonhee dont answer call called sent lot message past day hard reach guy like cant hang anyone must must mija fault immature say something wont speak mija freddie tell never want see yeonhee thanks concern yeonhee say something please let see dont touch two year later freddie andr shall play play dont normally play ill start first time seoul yes business trip five night six married three kid im going third divorce four kid came armament fair youre weapon dealer could say must one crazy world make crazy excuse gin tonic please theyre great two please sorry wine make sleepy match say like old guy turn youre artist student international consulting french company client korean yes way trojan horse boss exact word classic must know wanted fuck everything today know feeling go fuck everything hear music music coming inside love song hear ah thank im today hate birthday always ask question birthday question mother think somewhere cheer well excellent tell weapon weapon really interested yes weapon arent exciting part stake face usually end hostess bar morning men profession would brilliant yeah able look back booty call job interview shall go room dinner first dont time okay im anymore hey need pee thought youd back midnight oh sorry tinder date good day yeah fine finish yes want see sure beautiful kaykay love youre sure yeah im sure look tired okay need tenminute nap ill fine go surprise wait going know hate birthday know came please happy birthday really fooled earlier sorry thanks shes little sister lucie im little sister come okay think im little strung arent going open ill open tomorrow there lot people one work ask kaykay hope looking bottle absinthe cant find want help freddie ive got appointment hammond long three month told go right away told rush first meeting biological parent meeting told learn culture speak language talk adoptee come see looking toilet downstairs whats wrong mother responded biological mother six telegram two year today ten day ago letter addressed hammond said didnt want see short message twentytwo word thanks radish kimchi ate thank radish kimchi right thing yeonhee yeonhee korea want see dont drink like used youre id like see yeonhee saw talked france news paris korea want know happy birthday well well freddie andr already missing five year later maxime wear think dress formal youre overthinking go gut seriously prefer shit forgot ballet shoe idiot really havent seen seven year wear something comfortable thats come see thanks could text parent say weve arrived shouldnt love call andr okay well feeling okay far funny something huh youre even elegant paris go together tomorrow ill go ahead sort file korean okay wait assure know havent made decision one want force told deal tomorrow first meeting pressure yeah know pressure really like profile know missiledivision bos call james bond girl okay andr bye bye place toxic ill need dont worry im im serious maxime delicious happened france thailand weve got accident decided operation bangkok accident thailand tell nothing serious friend okay oh yeah scratch uh freddie driving scooter course look touch touch didnt tell u eat eat um im sorry restaurant dont know stop meat oh okay fine dont worry good really um delicious delicious yes thank thank dont drink alcohol really meat alcohol ecological reason um never felt good life yes alcohol stop alcohol stop alcohol stop okay alcohol stop okay tomorrow hell stop tomorrow tomorrow stop he changed since last time almost look younger weird look like photo showed think yes wish warned earlier pity youre coming gunsan possible came work work im mission uh ask job mm sell missile missile sell missile oh missile missile work french group collaborates many country like taiwan colombia help improve national defence system sell weapon make war war peace theory come korea sell missile actually freddie think maybe uh destiny destiny yeah left france baby shes adult shes coming back defend south korea north korea hm say yeonhees destiny help south korea north korea really father want play music wrote recently started play piano composes free time coincidence yeah weird bad someone selftaught im happy exchange normal text message happy health okay still drink lot uh drink le go looking taxi let go come taxi im sorry nice meet dont understand uh taxi bye bye hyehwa station bye bye normal guesthouse striking much resemble father take u itaewon area youre partying tonight yes going guesthouse changed mind could wipe life snap finger could wipe life snap finger hello frdrique benot sent message yes still seoul yes come come today jeonju abandoned old jeonju hammond another neighbourhood building destroyed hammond moved 2000s youre young original adoption file received copy seoul sure come never certain tone mother phone suggests come came early yes thats true yes ive brought hot tea thank ill explain follow mother arrives settled room next door le layout size one someone tell u arrived go speak mother separately prepare meeting like mother ready bring accept present meeting yes right yes mother arrived wait wait yes there something dont understand tell thought parent refused see child hammond could send telegram yes thats law accept request mother perhaps employee understand feeling adopted child freddie mother arrived daughter freddie touch yes one year later hello hello madam room available sure many night one night wait second single double room single room please passport please yeah happy birthday thank need wait bit u prepare room okay use bathroom sure take stair first floor mom sorry didnt write think im happy hope yeonhee new message subject reception address found return seoul translated banadera angelica l watch video online opensubtitles free browser extension osdblinkext"

In [20]:
cleaned_query = clean_subtitle(query)

# Calculate TF-IDF for the query
query_tfidf = tfidf_vectorizer.transform([cleaned_query])

# Calculate cosine similarity between the query and documents
cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix)

# Get the indices of documents sorted by similarity score
sorted_indices = cosine_similarities.argsort()[0][::-1]

In [22]:
# Return top 5 most similar chunk_text

top_k = 5
top_documents = chunks_df.loc[sorted_indices[:top_k], "chunk_text"].tolist()
print("Top 5 most similar chunk_text:")
for i, text in enumerate(top_documents, 1):
    print(f"Document {i}: {text}")

Top 5 most similar chunk_text:
Document 1: father want understand needed go somewhere youve always loved japan tokyo imagine surprise got message seoul japan longer option could called taken moment talk talk told plane tokyo cancelled due typhoon told go two week holiday next flight offered company seoul took really way want tell always said would go together yes know uh think good came alone must go mom im running credit already take care write u okay yes give dad kiss yes promise love darling dont understand telegram thing send third telegram there still reply hammond send least one year would risk complaint harassment send someone registered letter even sure right address im sorry possible leave two day probably never come back want mother address understand im sorry id really like rule protect parent child ask miss benot wish send third telegram mother address whats wrong come see paris yes well go lot region ill introduce friend well walk favourite part paris id like go jardin de 

In [23]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\prasad jadhav\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prasad jadhav\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Encode the chunk text to get semantic embeddings
chunk_embeddings = model.encode(chunks_df["chunk_text"].tolist())

# Add the embeddings to the DataFrame
chunks_df["embeddings"] = chunk_embeddings.tolist()

In [ ]:
import chromadb

# Create a client instance
client = chromadb.Client()

In [ ]:
collection = client.create_collection(
        name="searchengine",
        metadata={"hnsw:space": "cosine"} # l2 is the default
    )

In [ ]:
for i, embedding in enumerate(chunk_embeddings):
    collection.add(ids=f"chunk_{i}",  # Replace with a more descriptive ID scheme if needed
                   embeddings = embedding.tolist(),  # Convert to a list for Chroma
                   documents = chunks_df.loc[i, "chunk_text"],  # Use chunk_text as documents
                   metadatas = {"name": chunks_df.loc[i, "name"]}  # Use 'name' column data as metadata
    )

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load the SentenceTransformer Model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
subtitle_text = "also answer question invest something trendy"
cleaned_subtitle = clean_subtitle(subtitle_text)

In [ ]:
# Encode the cleaned subtitle text to get semantic embeddings
query_embedding = model.encode([cleaned_subtitle])

In [ ]:
# Perform a query on the ChromaDB collection using the embeddings
results = collection.query(
    query_embeddings=query_embedding,
    n_results=5,
    include=['documents']
)

In [ ]:
documents = results['documents']

# Iterate over the documents and print each document
for i, query_documents in enumerate(documents):
    for j, document in enumerate(query_documents):
        print(f'Document {i+1}, Item {j+1}: {document}')

In [ ]:
# More Advance Working Sonn..!
# Notebook Project By : PRASAD JADHAV (ML-ENG)
# LinkedIn: linkedin.com/in/prasadmjadhav2 | Github: github.com/prasadmjadhav2 | Mail: prasadmjadhav6161@gmail.com